# Information about the Data ( Example Data Set ) 
- Total RNA (TrueSeq Total RNA ribo zero )
   - Ribosomal RNA (rRNA) constitutes the majority (>98%) of total RNA preparations. To avoid wasting sequencing        reads, it is necessary to remove this abundant RNA before preparing RNA libraries for deep sequencing. https://www.nature.com/articles/nmeth.f.352 
- UHR (cancer) ERCC Mix 1    -- HBR (Brain) ERCC Mix 2 
- Spike-in: transcript of known sequence and quantity used to calibrate measurements in RNA hybridization assays, so a known quantity of spike-in is mixed with the experiment sample during preparation as an internal control of known sequence and quantity.
- read prefiltered to map Chr 22.
- Sequencing: paired end, Hiseq, 100bp.

####  Viewing the compressed file in the linux shell. 

In [22]:
zcat HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz | head

@HWI-ST718_146963544:7:2201:16660:89809/1
CAAAGAGAGAAAGAAAAGTCAATGATTTTATAGCCAGGCAAAATGACTTTCAAGTAAAAAATATAAAGCACCTTACAAACTAGTATCAAAATGCATTTCT
+
CCCFFFFFHHHHHJJJJJHIHIJJIJJJJJJJJJJJJIJJJJJJJJJJJJJIJJIIJJJJJJJJJJJJIIJFHHHEFFFFFEEEEEEEDDDDCDDEEDEE
@HWI-ST718_146963544:7:2215:16531:12741/1
CAAAATATTTTTTTTTTCTGTATATGACAAGACACACATCAGATCATAAGCTACAAGAAAACAAACAAAAAAGATATGAAAAAGATATAAAGACCTCCCC
+
@@@DDDDDFFFFFIIII;??::::9?99?G8;)9/8'787.)77;@==D=?;?A>D?@BDC@?CC=?BBBBB?<:4::@BBBB<?:>:@DD343<>:?BB
@HWI-ST718_146963544:8:1103:8309:14049/1
CACATTTCTCTGCATTTTCTCTTTACAGCAGTTTTTAAAATGTGTTTTGACCTTGTTTTTGCTATTATTTGGGATATTAAGTAAAGACATATTTTATGAG

gzip: stdout: Broken pipe


(Zcat) is a command line utility for viewing the contents of a compressed file without literally uncompressing it.
https://www.tecmint.com/linux-zcat-command-examples/

# Create a Reference Genome

In this case we are actually going to perform the analysis using only a single chromosome (chr22) and the ERCC spike-in to make it runs faster and to reduce the storage used capacity.

In [1]:
cd ~/GenomeIndices

In [ ]:
wget http://genomedata.org/rnaseq-tutorial/fasta/GRCh38/chr22_with_ERCC92.fa #download refrence genome (chr 22)

#### Reading the data content in the Reference Genome (Chr22).

In [5]:
cat chr22_with_ERCC92.fa | head -n 4

>22 dna_sm:chromosome chromosome:GRCh38:22:1:50818468:1 REF
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
NNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNNN
cat: write error: Broken pipe


- Purpose    : to view the first lines of the Reference Genome (chr22 in this case).
- Observation: Ns at the beginning and the ends (and usually at the centromere)
- Reason : these are highly repetitive region which cannot be recognized and most read alignments are built to know N ambiguity, so they will be ignored typically.
- Note    :Sometimes poly A tail comes from the spike-in like in this case but normally chromosome ends with Ns also.

#### Checking the number of lines and charachters (Basepairs).

In [31]:
wc chr22_with_ERCC92.fa 

  848761   848764 51751056 chr22_with_ERCC92.fa


- Purpose : to know the number of lines and characters are in this file and the length of this chromosome (in bases and Mb). 
- Observation: lines 848761, words 848764, characters(bp)51751056. 
- Note    :  ERCC92 stands for spike-in.

#### Display data from almost the middle of the Chr22 file. 

In [12]:
head -n 425000 chr22_with_ERCC92.fa | tail 

ggaggctgaggcaggagaatcgcttgaacatgggaggtggaagttgcagtgagccgaaac
tgcgccattgcactatagcctgggcaacaagagtgaaagtctgtcttgaaaaaaaaaaaT
CAGATGTTCTATGTAAAAATGCTATCTAtgattgaagtataaaactttacctccctttat
gttcctttgccctccccactatttattattgtcttgattatatcttctatatgcattgag
aggtgttataacttttgtatcaatcaccaaatttaatttagaaaatataagaggagaaga
aaagtctattacatttactcatatttttgcttactgtgttctttcttccttcttgatgtt
ccagaatttcttttattgcttcttttctgcttagaaaactttatctttttctttcatctt
tcttttttcctcctcctcctcctcctcctttttttttttttttttttttttttttttaat
aaagagacagggtctcactctatcacccagactggagttcagtgatgcaatcatagctca
ttgcaaccttgaactcctgggctcaagtgatcctcccacctcagcctcctgagtagctgg


# Getting the annotation

- annotation means getteing the  metadata file to combine it with the reference genome
- make it easy to deal with the Reference genome file (similar to the catalogue at the begning of the book )
- The annotation file is a description of where genetic element(intron, exon,..) located in the genome, in the form begin and end coordinate. https://www.biostars.org/p/99453/

In [ ]:
wget http://genomedata.org/rnaseq-tutorial/annotations/GRCh38/chr22_with_ERCC92.gtf

- notes:
  - in this pipeline we will use annotations obtained from Ensembl (Homo_sapiens.GRCh38.86.gtf.gz) for chromosome 22 only because we have just the Chr22 as our Refernce Genome as mentioned before.
  - (Genbank) database sometime contain both reference and annotation information.https://www.biostars.org/p/99453/

#### Checking the first lines of the annotation file :

In [5]:
 head -n2 chr22_with_ERCC92.gtf  #we can pipe this to less to view it more clearly

22	ensembl	gene	10736171	10736283	.	-	.	gene_id "ENSG00000277248"; gene_version "1"; gene_name "U2"; gene_source "ensembl"; gene_biotype "snRNA";
22	ensembl	transcript	10736171	10736283	.	-	.	gene_id "ENSG00000277248"; gene_version "1"; transcript_id "ENST00000615943"; transcript_version "1"; gene_name "U2"; gene_source "ensembl"; gene_biotype "snRNA"; transcript_name "U2.14-201"; transcript_source "ensembl"; transcript_biotype "snRNA"; tag "basic"; transcript_support_level "NA";


#### Retrieving the Start codon   

In [ ]:
less -p start_codon -S chr22_with_ERCC92.gtf 


22      ensembl start_codon     11066501        11066503        .       +       
22      ensembl exon    11067985        11068089        .       +       .       
22      ensembl CDS     11067985        11068086        .       +       0       
22      ensembl stop_codon      11068087        11068089        .       +       
22      havana  gene    11124337        11125705        .       +       .       
22      havana  transcript      11124337        11125705        .       +       
22      havana  exon    11124337        11124379        .       +       .       
22      havana  exon    11124508        11125705        .       +       .       
22      ensembl gene    11249809        11249959        .       -       .       
22      ensembl transcript      11249809        11249959        .       -       
22      ensembl exon    11249809        11249959        .       -       .       
22      havana  gene    12602466        12626642        .       +       .       
22      havana  transcript  

#### How many unique gene IDs are in the .gtf file?

An Ensembl gene (with a unique ENSG... ID) includes any spliced transcripts (ENST...) with overlapping coding sequence, with the exception of manually annotated readthrough genes which are annotated as a separate locus.
https://www.ensembl.org/info/genome/genebuild/genome_annotation.html

In [2]:
grep -o gene_id chr22_with_ERCC92.gtf | wc

  56295   56295  450360


This way didn't work because i got the count of all gene_ids rather than the unique_gene_id. 

#### perl command-line tool will solve this issue.

Perl is a general-purpose programming language originally developed for text manipulation . 
The output of this perl command will be a long list of ENSG Ids.
https://www.tutorialspoint.com/perl/perl_introduction.htm

In [3]:
perl -ne 'if ($_ =~ /(gene_id\s\"ENSG\w+\")/){print "$1\n"}' chr22_with_ERCC92.gtf | sort | uniq | wc -l

1318


In [ ]:
grep ENST00000342247 chr22_with_ERCC92.gtf | less -p "exon\s" -S

22      ensembl exon    17369366        17369909        .       +       .       
22      ensembl CDS     17369784        17369909        .       +       0       
22      ensembl start_codon     17369784        17369786        .       +       
22      ensembl exon    17477588        17477682        .       +       .       
22      ensembl CDS     17477588        17477682        .       +       0       
22      ensembl exon    17497403        17497586        .       +       .       
22      ensembl CDS     17497403        17497586        .       +       1       
22      ensembl exon    17499410        17499549        .       +       .       
22      ensembl CDS     17499410        17499549        .       +       0       
22      ensembl exon    17500631        17500735        .       +       .       
22      ensembl CDS     17500631        17500735        .       +       1       
22      ensembl exon    17503082        17503131        .       +       .       
22      ensembl CDS     1750

- Purpose : to view Now view the structure of a single transcript (ENST00000342247) in GTF format. 
- By piping to less and with the option -p we can pick information about exons in this script.

# Generating Index
use a computational strategy known as ‘indexing’ to speed up their mapping algorithms. Like the index at the end of a book, an index of a large DNA sequence allows one to rapidly find shorter sequences embedded within it.
https://www.biostars.org/p/212594/

In [ ]:
STAR --runMode genomeGenerate --runThreadN 2 --genomeDir ~/GenomeIndices --genomeFastaFiles ~/GenomeIndices/chr22_with_ERCC92.fa --sjdbGTFfile ~/GenomeIndices/chr22_with_ERCC92.gtf --sjdbOverhang 99

- Purpose: Depending on what samples we have sequenced (human, mouse etc.) we need to generate a Genome Index of the reference genome. Note: we do this once if we are going to work in the same species.
- Reason :  Genome indexing is one type pre processing to compress the size of text and to make queries fast.

In [2]:
samtools faidx ~/GenomeIndices/chr22_with_ERCC92.fa 

SAM Tools provide various utilities for manipulating alignments , including sorting, merging, indexing and generating alignments in a per-position format. http://samtools.sourceforge.net/

#### Creating a file with chromosome sizes.

In [5]:
cut -f1,2 ~/GenomeIndices/chr22_with_ERCC92.fa.fai > ~/GenomeIndices/hg38_chr22.chromosome.sizes

 - Purpose of generating a file with chromose sizes : 
  igv browser uses chrom.sizes files to define the chromosome lengths for a given genome. The file format is tab delimited, first column is chromosome name and second is its length. There can be more columns present, but they are ignored. https://software.broadinstitute.org/software/igv/chromSizes

In [1]:
cd ~/GenomeIndices

#### Generate a .fa file only containing the information of chr22 without the ERCC2 spike ins.

In [7]:
cat chr22_with_ERCC92.fa | perl -ne 'if ($_ =~ /\>22/){$chr22=1}; if ($_ =~ /\>ERCC/){$chr22=0}; if ($chr22){print "$_";}' > chr22_only.fa

# Obtaining RNA-Seq Data

In [4]:
cd ~/RNASeq/rawData

In [ ]:
wget http://genomedata.org/rnaseq-tutorial/HBR_UHR_ERCC_ds_5pc.tar

#### Unpack the .tar file ( the test data ) 

In [8]:
tar -xvf HBR_UHR_ERCC_ds_5pc.tar

HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz
HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz
HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz
HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz
HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz
HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz
UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz
UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz
UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz
UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz
UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz
UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz


- Observation : 
  - We have 6 pairs (12 files) because in fastq format, read 1 and read 2 of a each read pair (fragment) are stored in separate files.
- Notes :
  - The Linux “tar” stands for tape archive, which is used to deal with tape drives backup. 
  - The tar command used to rip a collection of files and directories into highly compressed archive file commonly called tarball or tar, gzip and bzip in Linux. https://www.tecmint.com/18-tar-command-examples-in-linux/

####  Viewing the first two read records of a file (in fastq format each read corresponds to 4 lines of data)

In [10]:
zcat UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz | head -n 8

@HWI-ST718_146963544:6:1213:8996:10047/1
CTTTTTTATTTTTGTCTGACTGGGTTGATTCAAAGGTCTGGTCTTTGAGCTCTTAAATTAGTTCTTCTATTTGGCCTAGTCTGTTGCTAAGGCTGCCAAC
+
CCCFFFFFHHHHGJHIIJHIHIIIFHIJJJJIJJGIBBFGEGGHIIHGGIJJIIHGGHIIIFGCGHHIIHIHHEEE?DFEFFFEEDCEEDDDDDDDBCDD
@HWI-ST718_146963544:5:2303:11793:37095/1
ATGAATTATAGGGCTGTATTTTAATTTTGCATTTTAAATTCCTGCAGTTTTCTTCCATCACTTTTCACCATGCATTGTATACTTGGAATTGCTTTTTGTG
+
@@??BDDFFF<FHEGFFGGIEBGHIIIIIBEHIIGIH<FHEFHHCHABF@DFHGGGII<DHBFGGGGBEGGIBHG@DHGIIIH@DE>CCHF:;>@BC>@@

gzip: stdout: Broken pipe


#### Checking the number of reads in the first library 

In [12]:
zcat UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz | grep -P "^\@HWI" | wc -l

227392


- we use zcat to view the contents of a compressed then piping the results to the grep command.
- GNU grep has a -P option (perl) that can be used
- we pipe the output of grep -P as standard output to wc -l 'wc' to do a word count ('-l' gives lines). 

# Performing quality check with the obtained data and inspect the results.

- we can use FastQC to get a sense of the data quality before alignment.
- FastQC aims to provide a simple way to do some quality control checks on raw sequence data coming from high throughput sequencing pipelines.
- Summary graphs and tables to quickly assess your data http://www.bioinformatics.babraham.ac.uk/projects/fastqc/

#### Performing quality check in parallel to speed up the process

In [ ]:
find . -name "*.fastq.gz" | parallel fastqc -o {//}/ {}

- Notes
  - GNU parallel is a shell tool for executing jobs in parallel http://www.gnu.org/software/parallel/
  - FastQC can do multiple files at once, in multithreaded mode Either use the -t option (see fastqc --help) or add a     single CPU for each file, so they can be launched parallel https://github.com/SciLifeLab/Sarek/issues/631

In [14]:
cd ~/RNASeq/rawData

# Adapter trimming

 - Illumina FASTQ file generation pipelines include an adapter trimming option for the removal of adapter sequences from the 3’ ends of reads.
#### Why should adapter sequence be removed?
   - Adapter sequences should be removed from reads because they interfere with downstream analyses, such as alignment of reads to a reference.
   - The adapters contain the sequencing primer binding sites, the index sequences, and the sites that allow library fragments to attach to the flow cell lawn. Libraries prepared with Illumina library prep kits require adapter trimming only on the 3’ ends of reads, because adapter sequences are not found on the 5’ ends.
 https://support.illumina.com/bulletins/2016/04/adapter-trimming-why-are-adapter-sequences-trimmed-from-only-the--ends-of-reads.html

In [15]:
mkdir ~/RNASeq/rawDataTrim

mkdir: cannot create directory '/home/mlsi/RNASeq/rawDataTrim': File exists


: 1

In [1]:
cd ~/RNASeq/rawDataTrim

In [ ]:
wget http://genomedata.org/rnaseq-tutorial/illumina_multiplex.fa #this step was already done

#### Perform Adapter trimming with all .fastq.gz in the following way:

In [18]:
ls *.fastq.gz

HBR_1_1.fastq.gz  HBR_2_2.fastq.gz  UHR_1_1.fastq.gz  UHR_2_2.fastq.gz
HBR_1_2.fastq.gz  HBR_3_1.fastq.gz  UHR_1_2.fastq.gz  UHR_3_1.fastq.gz
HBR_2_1.fastq.gz  HBR_3_2.fastq.gz  UHR_2_1.fastq.gz  UHR_3_2.fastq.gz


### Flexbar – flexible barcode and adapter removal


In [7]:
flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters ~/RNASeq/rawDataTrim/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 2 --zip-output GZ --reads ~/RNASeq/rawData/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 ~/RNASeq/rawData/HBR_Rep1_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz --target ~/RNASeq/rawDataTrim/HBR_1

In [20]:
flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters ~/RNASeq/rawDataTrim/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 2 --zip-output GZ --reads ~/RNASeq/rawData/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 ~/RNASeq/rawData/HBR_Rep2_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz --target ~/RNASeq/rawDataTrim/HBR_2

In [21]:
flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters ~/RNASeq/rawDataTrim/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 2 --zip-output GZ --reads ~/RNASeq/rawData/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 ~/RNASeq/rawData/HBR_Rep3_ERCC-Mix2_Build37-ErccTranscripts-chr22.read2.fastq.gz --target ~/RNASeq/rawDataTrim/HBR_3

In [22]:
flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters ~/RNASeq/rawDataTrim/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 2 --zip-output GZ --reads ~/RNASeq/rawData/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 ~/RNASeq/rawData/UHR_Rep1_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz --target ~/RNASeq/rawDataTrim/UHR_1

In [23]:
flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters ~/RNASeq/rawDataTrim/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 2 --zip-output GZ --reads ~/RNASeq/rawData/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 ~/RNASeq/rawData/UHR_Rep2_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz --target ~/RNASeq/rawDataTrim/UHR_2

In [24]:
flexbar --adapter-min-overlap 7 --adapter-trim-end RIGHT --adapters ~/RNASeq/rawDataTrim/illumina_multiplex.fa --pre-trim-left 13 --max-uncalled 300 --min-read-length 25 --threads 2 --zip-output GZ --reads ~/RNASeq/rawData/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read1.fastq.gz --reads2 ~/RNASeq/rawData/UHR_Rep3_ERCC-Mix1_Build37-ErccTranscripts-chr22.read2.fastq.gz --target ~/RNASeq/rawDataTrim/UHR_3

- Notes:
- The program Flexbar preprocesses high-throughput sequencing data efficiently. 
- It demultiplexes barcoded runs and removes adapter sequences. 
- Several adapter removal presets for Illumina libraries are included. 
- Flexbar computes exact overlap alignments using SIMD and multicore parallelism.
https://github.com/seqan/flexbar

'--adapter-min-overlap 7' requires a minimum of 7 bases to match the adapter
'--adapter-trim-end RIGHT' uses a trimming strategy to remove the adapter from the 3 prime or RIGHT end of the read
https://github.com/griffithlab/rnaseq_tutorial/wiki/Adapter-Trim

# STAR Alignment

- STAR Aligner
  - To determine where on the human genome our reads originated from, we will align our reads to the reference genome using STAR (Spliced Transcripts Alignment to a Reference). STAR is an aligner designed to specifically address many of the challenges of RNA-seq data mapping using a strategy to account for spliced alignments.
- STAR Alignment Strategy
  - STAR is shown to have high accuracy and outperforms other aligners by more than a factor of 50 in mapping speed, but it is memory intensive. The algorithm achieves this highly efficient mapping by performing a two-step process:
    - Seed searching     - Clustering, stitching, and scoring.
https://hbctraining.github.io/Intro-to-rnaseq-hpc-O2/lessons/03_alignment.html

#### Checking the mapping folder.

In [41]:
cd ~/RNASeq/mapping

#### Creating a mapping command 

In [39]:
cd ~/RNASeq/rawDataTrim

In [42]:
ls

'*.bam'   HBR_1   HBR_2   HBR_3   SampleNames.txt   UHR_1   UHR_2   UHR_3


In [24]:
parallel -j 1 echo {1}_{2} ::: UHR HBR ::: 1 2 3 > SampleNames.txt

- Basic Usage of STAR:
 #STAR --genomeDir --readFilesIn *_1.fastq.gz *_2.fastq.gz --outSAMattributes All --runThreadN --readFilesCommand zcat --outStd SAM | samtools sort > ~/RNASeq/mapping/*.bam"

In [29]:
cat /home/mlsi/RNASeq/rawDataTrim/SampleNames.txt | parallel "mkdir ~/RNASeq/mapping/{}; cd ~/RNASeq/mapping/{} ; STAR --genomeDir /home/mlsi/GenomeIndices --readFilesIn /home/mlsi/RNASeq/rawDataTrim/{}_1.fastq.gz /home/mlsi/RNASeq/rawDataTrim/{}_2.fastq.gz --outSAMattributes All --runThreadN 2 --readFilesCommand zcat --outStd SAM | samtools sort > ~/RNASeq/mapping/{}/{}.bam"

mkdir: cannot create directory '/home/mlsi/RNASeq/mapping/UHR_2': File exists
samtools sort: couldn't allocate memory for bam_mem
mkdir: cannot create directory '/home/mlsi/RNASeq/mapping/UHR_1': File exists
mkdir: cannot create directory '/home/mlsi/RNASeq/mapping/UHR_3': File exists
samtools sort: couldn't allocate memory for bam_mem
mkdir: cannot create directory '/home/mlsi/RNASeq/mapping/HBR_1': File exists
mkdir: cannot create directory '/home/mlsi/RNASeq/mapping/HBR_2': File exists
samtools sort: couldn't allocate memory for bam_mem
mkdir: cannot create directory '/home/mlsi/RNASeq/mapping/HBR_3': File exists


: 3

- note:
  - This problem " couldn't allocate memory for bam_mem " was solved by increasing the RAM of the Virutal machine.

# Data Visualization 

## Visualization of the genomic data using IGV browser . 
- IGV is a desktop application for the visualization and interactive exploration of genomic data in the context of a reference genome. A key characteristic of IGV is its focus on the integrative nature of genomic studies. It allows investigators to flexibly visualize many different types of data together—and importantly also integrate these data with the display of sample attribute information such as clinical and phenotypic information. (https://www.ncbi.nlm.nih.gov/pmc/articles/PMC3603213/)
- notes :  
  - sam and bam files are used then in visualization. 
  - exons are depicted  in blue bars and the gaps in between are the introns.
  - igv browser need a narrow window to detect these reads that’s why we zoom in deep. 
  - how many reads which cover one specific region ( arrow down in viewing  alignments ).
  - if you found difference in just one read then mostly would be an error. 

In [ ]:
Length: 116602880 (111M) [application/x-tar]cd HBR_1

In [ ]:
ll

In [ ]:
cat /home/mlsi/RNASeq/rawDataTrim/SampleNames.txt| parallel --eta --verbose "cd ~ /RNASeq/mapping/{}; find . -name "{}.bam" samtools index {}.bam"

In [ ]:
cd~/RNASeq/mapping/UHR_1, find . -name UHR_1.bam; samtools index UHR_1.bam
cd~/RNASeq/mapping/UHR_2, find . -name UHR_1.bam; samtools index UHR_2.bam

- notes
  - cigar save a lot of storage and time 87M for xample stands for 87 matches give you a hint matches and mismatches       and insertion and deletion.
  - serching for mutaions via igv browser is not the best way.
  - from bam files we do not get which insertion or deletion  but yes or no 

## Formats other .bam files
 - The bigWig files are in an indexed binary format. 
 - The main advantage of this format is that only those portions of the file needed to display a particular region are    transferred to the Genome Browser server.
 - Because of this, bigWig files have considerably faster display performance than regular wiggle files when working      with large data sets.https://genome.ucsc.edu/goldenPath/help/bigWig.html

In [ ]:
conda install ucsc-bedgraphtobigwig

#### convert .bam files to bedgraphs

In [ ]:
cat /home/mlsi/RNASeq/rawDataTrim/SampleNames.txt| parallel --eta--verbose "cd ~/RNASeq/mapping/{} ; bedtools genomecov -ibam {}.bam -g~/GenomeIndices/hg38_chr22.chromosome.sizes -bg | sort -k1,1 -k2,2n >{.}.bedgraph"

#### convert bedgraphs to bidwig

In [ ]:
cat /home/mlsi/RNASeq/rawDataTrim/SampleNames.txt| parallel --eta--verbose "cd ~/RNASeq/mapping/{}; bedGraphToBigWig {}.bedgraph~/GenomeIndices/hg38_chr22.chromosome.sizes/home/mlsi/RNASeq/bigWig/{}.bigwig"`

# Count Table

After getting the reads which are alligned to the genome, then we should count how many reads have mapped to each gene.in other words : Given mapped reads in a BAM file and gene locations in a GTF file, then calculates how many reads map to each gene. One of the most popular tools to do this are featureCounts for assigning sequence reads to genomic features. also,HTSeq using own GTF (https://chipster.csc.fi/manual/htseq-count-own-gtf.html). 

In [ ]:
featureCounts -a ~/GenomeIndices/chr22_with_ERCC92.gtf -g gene_name -o /home/mlsi/RNASeq/countTable/featureCounts.txt ~/RNASeq/mapping/*/*.bam

- featureCounts:
  -  program suitable for counting reads generated from either RNA or genomic DNA sequencing experiments. 
  -  implements highly efficient chromosome hashing and feature blocking techniques. 
  -  It is considerably faster than existing methods (by an order of magnitude for gene-level summarization) and   requires far less computer memory. 
  - It works with either single or paired-end reads and provides a wide range of options appropriate for different  sequencing applications. (https://www.ncbi.nlm.nih.gov/pubmed/24227677)

In [7]:
cd /home/mlsi/RNASeq/countTable

In [3]:
ls

featureCounts.txt  featureCounts.txt.summary


In [4]:
cat featureCounts.txt | head -2

# Program:featureCounts v1.6.4; Command:"featureCounts" "-a" "/home/mlsi/GenomeIndices/chr22_with_ERCC92.gtf" "-g" "gene_name" "-o" "/home/mlsi/RNASeq/countTable/featureCounts.txt" "/home/mlsi/RNASeq/mapping/HBR_1/HBR_1.bam" "/home/mlsi/RNASeq/mapping/HBR_2/HBR_2.bam" "/home/mlsi/RNASeq/mapping/HBR_3/HBR_3.bam" "/home/mlsi/RNASeq/mapping/UHR_1/UHR_1.bam" "/home/mlsi/RNASeq/mapping/UHR_2/UHR_2.bam" "/home/mlsi/RNASeq/mapping/UHR_3/UHR_3.bam" 
Geneid	Chr	Start	End	Strand	Length	/home/mlsi/RNASeq/mapping/HBR_1/HBR_1.bam	/home/mlsi/RNASeq/mapping/HBR_2/HBR_2.bam	/home/mlsi/RNASeq/mapping/HBR_3/HBR_3.bam	/home/mlsi/RNASeq/mapping/UHR_1/UHR_1.bam	/home/mlsi/RNASeq/mapping/UHR_2/UHR_2.bam	/home/mlsi/RNASeq/mapping/UHR_3/UHR_3.bam
cat: write error: Broken pipe


- In this case, the counts.txt file will contain a column for each of the samples, for a total of 13 columns. 
- The column names are : Geneid, Chr, Start, End, Strand Length,...	

In [8]:
cat featureCounts.txt | sort -rn -k 7 | head -3

ERCC-00002	ERCC-00002	1	1061	+	1061	35528	43602	39442	38532	24086	31668
ERCC-00074	ERCC-00074	1	522	+	522	19560	22912	21320	30074	21414	24410
ERCC-00096	ERCC-00096	1	1107	+	1107	19444	23814	21058	42538	26930	33888
sort: write failed: 'standard output': Broken pipe
sort: write error


- Observation : the counts in the last column correlate very closely with the expected abundances for Mix2 in the ECC Mix file.

#### Now we will proceed with our pipeline in Rstudio